# Notebook Configuration

In [1]:
import sys
sys.path.append('/home/rodari78/tesis/CNN_bike_lanes/')

import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle

from libs.YOLO.preprocessing import *
from libs.YOLO.utils import *
from ultralytics import YOLO

# change base dir
os.chdir('/home/rodari78/tesis/CNN_bike_lanes/app')

Declare paths and constants

In [2]:
# dataset paths
PRM_PATH_ROOT = 'datasets'

# main files
PRM_COUNTRY_LIST = ['China_MotorBike',
                    'China_Drone',
                    'Czech',
                    'India',
                    'Japan',
                    'Norway',
                    'United_States']

# run version
PRM_YOLO_VERSION = 7
PRM_YOLO_VERSION = str(PRM_YOLO_VERSION).zfill(2)

# get data
PRM_DATA_VERSION = 5
PRM_DATA_VERSION = str(PRM_DATA_VERSION).zfill(3)
PRM_DATA_SUBSETS = f'datasets_track/subsets_v{PRM_DATA_VERSION}.csv'

PRM_IMG_INPUT_PATH = 'datasets/lima_bike_lanes/images/test'
PRM_LBL_INPUT_PATH = 'datasets/lima_bike_lanes/labels/test'

PRM_IMG_OUTPUT_ROOT_PATH = 'datasets/lima_bike_lanes_subsets/images'
PRM_LBL_OUTPUT_ROOT_PATH = 'datasets/lima_bike_lanes_subsets/labels'

# config file
PRM_YML_FILE = 'yaml/config_v004.yaml'

# evaluation yamls
PRM_YAML_TRAIN = 'yaml/train_subset.yaml'
PRM_YAML_VAL = 'yaml/val_subset.yaml'
PRM_YAML_TEST = 'yaml/test_subset.yaml'

# get pretrained model
PRM_IS_CAL = True

PRM_WEIGHT_CAL = 'runs/detect/version_02/train/weights/best.pt'

PRM_PRE_MODEL_VERSION = '11'
PRM_PRE_MODEL_SIZE = 's'
PRM_PRE_MODEL_SIZE = PRM_PRE_MODEL_SIZE.lower()

PRM_PRE_MODEL_PATH = f'pretrained_models/YOLO/v{PRM_PRE_MODEL_VERSION}/yolo{PRM_PRE_MODEL_VERSION}{PRM_PRE_MODEL_SIZE}.pt' if not PRM_IS_CAL else PRM_WEIGHT_CAL

# declare paths
PRM_BASE_PATH = 'runs/detect'

PRM_VERSION_NAME = f'version_{PRM_YOLO_VERSION}'

PRM_FITTING_PATH = f'{PRM_BASE_PATH}/{PRM_VERSION_NAME}'

PRM_WHOLE_FITTING_PATH = f'/home/rodari78/tesis/CNN_bike_lanes/app/{PRM_FITTING_PATH}'
PRM_WHOLE_OBJECT_PATH = f'{PRM_WHOLE_FITTING_PATH}/objects'

os.makedirs(PRM_FITTING_PATH, exist_ok = True)
os.makedirs(PRM_WHOLE_OBJECT_PATH, exist_ok = True)

PRM_BEST_MODEL_PATH = f'{PRM_FITTING_PATH}/train/weights/best.pt'

PRM_TRAIN_METRICS_NAME = f'version_{PRM_YOLO_VERSION}/train_metrics'
PRM_VAL_METRICS_NAME = f'version_{PRM_YOLO_VERSION}/val_metrics'
PRM_TEST_METRICS_NAME = f'version_{PRM_YOLO_VERSION}/test_metrics'
PRM_PREDICTION_NAME = f'version_{PRM_YOLO_VERSION}/predictions'

PRM_TRAIN_METRICS_PATH = f'{PRM_BASE_PATH}/{PRM_TRAIN_METRICS_NAME}'
PRM_VAL_METRICS_PATH = f'{PRM_BASE_PATH}/{PRM_VAL_METRICS_NAME}'
PRM_TEST_METRICS_PATH = f'{PRM_BASE_PATH}/{PRM_TEST_METRICS_NAME}'
PRM_PREDICTION_NAME_PATH = f'{PRM_BASE_PATH}/{PRM_PREDICTION_NAME}'

PRM_PREDICTIONS_SOURCE = 'datasets/lima_bike_lanes/images/test'

# object path
PRM_TRAIN_METRICS_OBJ = f'{PRM_WHOLE_OBJECT_PATH}/train_results.pkl'
PRM_VAL_METRICS_OBJ = f'{PRM_WHOLE_OBJECT_PATH}/val_results.pkl'
PRM_TEST_METRICS_OBJ = f'{PRM_WHOLE_OBJECT_PATH}/test_results.pkl'
PRM_ALL_METRICS_OBJ = f'{PRM_WHOLE_OBJECT_PATH}/results.pkl'

Display summary

In [8]:
summary = pd.Series({'Version': PRM_YOLO_VERSION,
                     'Data subsets': PRM_DATA_SUBSETS,
                     'Training yaml': PRM_YML_FILE,
                     'Fitting path': PRM_FITTING_PATH,
                     'Eval training yaml': PRM_YAML_TRAIN,
                     'Eval validation yaml': PRM_YAML_VAL,
                     'Eval testing yaml': PRM_YAML_TEST,
                     'Predictions': PRM_PREDICTIONS_SOURCE,
                     'Input model': PRM_PRE_MODEL_PATH,
                     'Output model': PRM_BEST_MODEL_PATH})\
            .reset_index()\
            .rename(columns = {'index': 'Description',
                               0: 'Values'})

display(summary)

,Description,Values
0,Version,07
1,Data subsets,datasets_track/subsets_v005.csv
2,Training yaml,yaml/config_v004.yaml
3,Fitting path,runs/detect/version_07
4,Eval training yaml,yaml/train_subset.yaml
5,Eval validation yaml,yaml/val_subset.yaml
6,Eval testing yaml,yaml/test_subset.yaml
7,Predictions,datasets/lima_bike_lanes/images/test
8,Input model,runs/detect/version_02/train/weights/best.pt
9,Output model,runs/detect/version_07/train/weights/best.pt


Clean config

In [8]:
# create needed paths for each country
GLOB_COUNTRY_PATH = {country: (f'{PRM_PATH_ROOT}/init_base{country}/images/train', f'{PRM_PATH_ROOT}/init_base{country}/images/val',
                               f'{PRM_PATH_ROOT}/init_base{country}/labels/train', f'{PRM_PATH_ROOT}/init_base{country}/labels/val')
                               for country in PRM_COUNTRY_LIST}
# clean files
clean_files(GLOB_COUNTRY_PATH)

Country: China_MotorBike
Country: China_Drone
Country: Czech
Country: India
Country: Japan
Country: Norway
Country: United_States


Distribute images

In [4]:
# get data subsets
image_subsets = pd.read_csv(PRM_DATA_SUBSETS)

distribute_images(images_df = image_subsets, 
                  img_in_path = PRM_IMG_INPUT_PATH, 
                  lbl_in_path = PRM_LBL_INPUT_PATH, 
                  img_out_path_root = PRM_IMG_OUTPUT_ROOT_PATH, 
                  lbl_out_ath_root = PRM_LBL_OUTPUT_ROOT_PATH, 
                  subsets = ['train', 'val', 'test'])

Fit pretrained model

In [5]:
model = YOLO(PRM_PRE_MODEL_PATH)
results = model.train(data = PRM_YML_FILE, 
                      epochs = 50, 
                      save = True, 
                      label_smoothing = 0.1,
                      batch = 16,
                      workers = 48,
                      imgsz = 640,
                      seed = 123,
                      iou = 0.7,
                      name = f'{PRM_VERSION_NAME}/train')

New https://pypi.org/project/ultralytics/8.3.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A4000, 16376MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/version_02/train/weights/best.pt, data=yaml/config_v004.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=48, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=123, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

train: Scanning /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/labels/train... 1394 images, 578 backgrounds, 2 corrupt: 100%|██████████| 1394/1394 [00:09<00:00, 151.75it/s]

train: WARNING ⚠️ /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/images/train/img_00459.png: 1 duplicate labels removed
train: WARNING ⚠️ /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/images/train/img_00572.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2861]
train: WARNING ⚠️ /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/images/train/img_00715.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0791]
train: New cache created: /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/labels/train.cache



val: Scanning /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/labels/val... 356 images, 151 backgrounds, 0 corrupt: 100%|██████████| 356/356 [00:02<00:00, 151.39it/s]

val: New cache created: /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/labels/val.cache


...

      50/50       4.3G      1.105     0.7724      1.084         18        640: 100%|██████████| 87/87 [00:14<00:00,  5.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:02<00:00,  5.54it/s]

                   all        356        638      0.602       0.48      0.486      0.266



50 epochs completed in 0.243 hours.
Optimizer stripped from /home/rodari78/tesis/CNN_bike_lanes/app/runs/detect/version_07/train/weights/last.pt, 19.2MB
Optimizer stripped from /home/rodari78/tesis/CNN_bike_lanes/app/runs/detect/version_07/train/weights/best.pt, 19.2MB

Validating /home/rodari78/tesis/CNN_bike_lanes/app/runs/detect/version_07/train/weights/best.pt...
Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A4000, 16376MiB)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:08<00:00,  1.41it/s]


                   all        356        638       0.66      0.464      0.501      0.268
D00 - Longitudinal Crack        102        193      0.601      0.576      0.577       0.36
D10 - Transverse Crack        116        210      0.634      0.443      0.476      0.249
 D20 - Alligator Crack         36         54      0.705      0.407      0.478      0.234
  D40 - Potholes Crack         80        181        0.7      0.431      0.473      0.231
Speed: 0.1ms preprocess, 3.5ms inference, 0.0ms loss, 8.1ms postprocess per image
Results saved to /home/rodari78/tesis/CNN_bike_lanes/app/runs/detect/version_07/train


# Evaluate Model

**TRAINING SET**

In [ ]:
model = YOLO(PRM_BEST_MODEL_PATH)
if os.path.exists(PRM_TRAIN_METRICS_PATH): shutil.rmtree(PRM_TRAIN_METRICS_PATH)
results = model.val(data = PRM_YAML_TRAIN, 
                    name = PRM_TRAIN_METRICS_NAME)

# parse results
results_dict = parse_yolo_results(results)
results_parsed = ParseYOLOResults(**results_dict)

# save results
save_object(results_parsed, PRM_TRAIN_METRICS_OBJ)

# load results
results_parsed = load_object(PRM_TRAIN_METRICS_OBJ)

results_parsed.results_dict

Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A4000, 16376MiB)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/labels/train.cache... 1394 images, 578 backgrounds, 2 corrupt: 100%|██████████| 1394/1394 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/images/train/img_00459.png: 1 duplicate labels removed
train: WARNING ⚠️ /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/images/train/img_00572.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2861]
train: WARNING ⚠️ /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/images/train/img_00715.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0791]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 87/87 [00:10<00:00,  8.40it/s]


                   all       1392       2722      0.884      0.832      0.905      0.617
D00 - Longitudinal Crack        408        958      0.903      0.917      0.953      0.725
D10 - Transverse Crack        460        882      0.851      0.804       0.88      0.568
 D20 - Alligator Crack        146        198        0.9      0.808      0.899       0.62
  D40 - Potholes Crack        315        684      0.882      0.798      0.886      0.556
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/rodari78/tesis/CNN_bike_lanes/app/runs/detect/version_07/train_metrics


{'metrics/precision(B)': 0.8841501994820047,
 'metrics/recall(B)': 0.831907936864252,
 'metrics/mAP50(B)': 0.9045993135354256,
 'metrics/mAP50-95(B)': 0.6173142877832248,
 'fitness': 0.6460427903584449}

**VALIDATION SET**

In [7]:
model = YOLO(PRM_BEST_MODEL_PATH)
if os.path.exists(PRM_VAL_METRICS_PATH): shutil.rmtree(PRM_VAL_METRICS_PATH)
results = model.val(data = PRM_YAML_VAL,
                    name = PRM_VAL_METRICS_NAME)

# parse results
results_dict = parse_yolo_results(results)
results_parsed = ParseYOLOResults(**results_dict)

# save results
save_object(results_parsed, PRM_VAL_METRICS_OBJ)

# load results
results_parsed = load_object(PRM_VAL_METRICS_OBJ)

results_parsed.results_dict

Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A4000, 16376MiB)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/labels/val.cache... 356 images, 151 backgrounds, 0 corrupt: 100%|██████████| 356/356 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:03<00:00,  7.52it/s]


                   all        356        638      0.661      0.464      0.501       0.27
D00 - Longitudinal Crack        102        193      0.599      0.574      0.578      0.361
D10 - Transverse Crack        116        210      0.639      0.443      0.476      0.249
 D20 - Alligator Crack         36         54      0.707      0.407      0.478      0.237
  D40 - Potholes Crack         80        181      0.699      0.431      0.474      0.234
Speed: 0.3ms preprocess, 4.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /home/rodari78/tesis/CNN_bike_lanes/app/runs/detect/version_07/val_metrics


{'metrics/precision(B)': 0.6612151655188693,
 'metrics/recall(B)': 0.46367625337228324,
 'metrics/mAP50(B)': 0.5012546498317669,
 'metrics/mAP50-95(B)': 0.27035109439642835,
 'fitness': 0.29344144993996224}

**TESTING SET**

In [8]:
model = YOLO(PRM_BEST_MODEL_PATH)
if os.path.exists(PRM_TEST_METRICS_PATH): shutil.rmtree(PRM_TEST_METRICS_PATH)
results = model.val(data = PRM_YAML_TEST,
                    imgsz = 640,
                    name = PRM_TEST_METRICS_NAME)

# parse results
results_dict = parse_yolo_results(results)
results_parsed = ParseYOLOResults(**results_dict)

# save results
save_object(results_parsed, PRM_TEST_METRICS_OBJ)

# load results
results_parsed = load_object(PRM_TEST_METRICS_OBJ)

results_parsed.results_dict

Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A4000, 16376MiB)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes_subsets/labels/test.cache... 750 images, 310 backgrounds, 0 corrupt: 100%|██████████| 750/750 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:05<00:00,  8.13it/s]


                   all        750       1476      0.641      0.448      0.498      0.272
D00 - Longitudinal Crack        220        512      0.614      0.544      0.552       0.33
D10 - Transverse Crack        247        453      0.675      0.461      0.523      0.267
 D20 - Alligator Crack         80        115      0.619        0.4      0.463      0.257
  D40 - Potholes Crack        171        396      0.658      0.386      0.455      0.232
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/rodari78/tesis/CNN_bike_lanes/app/runs/detect/version_07/test_metrics


{'metrics/precision(B)': 0.641492903303228,
 'metrics/recall(B)': 0.4479027039546799,
 'metrics/mAP50(B)': 0.4983044115142238,
 'metrics/mAP50-95(B)': 0.2715568186083496,
 'fitness': 0.29423157789893706}

**JOIN RESULTS**

In [12]:
# get pickle files
results_pkl = {obj.split('_')[0]: load_object(f'{PRM_WHOLE_OBJECT_PATH}/{obj}') for obj in os.listdir(PRM_WHOLE_OBJECT_PATH) if obj.split('_')[0] in ['train', 'val', 'test']}

# save into wrapper
results_all = YOLOResults(train = results_pkl['train'],
                          val = results_pkl['val'],
                          test = results_pkl['test'])

save_object(results_all, PRM_ALL_METRICS_OBJ)

# read object
results_all = load_object(PRM_ALL_METRICS_OBJ)
results_all

YOLOResults(train=ParseYOLOResults(ap_class_index=array([0, 1, 2, 3]), box=ParseYOLOBox(all_ap=array([[    0.95318,     0.94772,     0.93789,     0.91715,       0.894,     0.84894,     0.74097,     0.59343,      0.3492,    0.068001],
       [    0.88002,     0.87102,     0.85312,     0.82775,     0.76495,     0.65165,     0.46855,     0.26502,    0.088197,   0.0086981],
       [    0.89946,     0.89176,     0.88036,     0.86168,     0.81752,     0.74761,     0.56455,     0.35357,     0.16322,    0.018774],
       [    0.88573,     0.86698,     0.84878,      0.7994,     0.73536,     0.61521,     0.45449,     0.26213,    0.090659,    0.005864]]), ap=array([    0.72505,      0.5679,     0.61985,     0.55646]), ap50=array([    0.95318,     0.88002,     0.89946,     0.88573]), ap_class_index=array([0, 1, 2, 3]), curves=[], curve_results=[[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0

Make predictions

In [3]:
# load weights
model = YOLO(PRM_BEST_MODEL_PATH)
if os.path.exists(PRM_PREDICTION_NAME_PATH): shutil.rmtree(PRM_PREDICTION_NAME_PATH)
# validate on testing set (Lima bikelanes) with yolo
results = model.predict(source = 'datasets/lima_bike_lanes/images/test',
                        data = 'yaml/test.yaml',
                        name = PRM_PREDICTION_NAME,
                        imgsz = 640, 
                        save = True, 
                        save_txt = True,
                        show = False)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/11000 /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes/images/test/img_00000.png: 640x640 (no detections), 8.4ms
image 2/11000 /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes/images/test/img_00001.png: 640x640 (no detections), 7.9ms
image 3/11000 /home/rodari78/tesis/CNN_bike_lanes/app/datasets/lima_bike_lanes/images/test/img_00002.png: 640x640 (no detections), 8.8ms
image 4/11000 /home/rodari78/tesis/CNN_bike_l